# Композиции алгоритмов регрессии

## Необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso

## Подготовка данных

In [2]:
path_to_file = '../datasets/regression/metro_interstate_traffic_volume_preprocessed.csv'
dataset = pd.read_csv(path_to_file)

In [3]:
dataset.head()

,Unnamed: 0,temp,rain_1h,snow_1h,clouds_all,date_time,traffic_volume,h_Christmas Day,h_Columbus Day,h_Independence Day,...,wd_sleet,wd_smoke,wd_snow,wd_thunderstorm,wd_thunderstorm with drizzle,wd_thunderstorm with heavy rain,wd_thunderstorm with light drizzle,wd_thunderstorm with light rain,wd_thunderstorm with rain,wd_very heavy rain
0,0,288.28,0.0,0.0,40,2012-10-02 09:00:00,5545,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,289.36,0.0,0.0,75,2012-10-02 10:00:00,4516,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,289.58,0.0,0.0,90,2012-10-02 11:00:00,4767,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,290.13,0.0,0.0,90,2012-10-02 12:00:00,5026,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,291.14,0.0,0.0,75,2012-10-02 13:00:00,4918,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Удалим ненужные столбцы, разобьём выборку и отмасштабируем

In [4]:
dataset = dataset.drop(['Unnamed: 0', 'date_time'], axis=1)

In [5]:
X = dataset.iloc[:, [0, 1, 2, 3] + list(range(5, 65))]
y = dataset.iloc[:, 4]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Композиции алгоритмов

Сначала обучим базовый алгоритм, измерим время. Затем обучим композиции алгоритмов с подбором гиперпараметров и измерим время

### Базовые алгоритмы

In [12]:
%%time
dtr = DecisionTreeRegressor()

params = {
    'max_depth': list(range(5, 16)),
    'min_samples_leaf': [1, 3, 5],
    'max_features': ['auto', 'sqrt', 'log2']
}

reg = GridSearchCV(dtr, params)

reg.fit(X_train, y_train)

print(reg.score(X_test, y_test))
print(reg.best_params_)

0.07195975073899563
{'max_depth': 7, 'max_features': 'auto', 'min_samples_leaf': 5}
Wall time: 38.7 s


### Бэггинг

**Бэггинг**: каждый базовый алгоритм обучается на подвыборке исходной обучающей выборки. Формирование подвыборке происходит при помощи *бутстрапа* (выбор с возвращением элементов из исходного множества)

In [27]:
bagging_tree_reg = BaggingRegressor(DecisionTreeRegressor(random_state=42))

In [28]:
params = {
    'n_estimators': [10, 20, 30, 40, 50],
    'max_features': [0.8, 0.9, 1.0]
}

In [32]:
%%time
reg = GridSearchCV(bagging_tree_reg, params)

reg.fit(X_train, y_train)

Wall time: 6min 31s


GridSearchCV(estimator=BaggingRegressor(base_estimator=DecisionTreeRegressor(random_state=42)),
             param_grid={'max_features': [0.8, 0.9, 1.0],
                         'n_estimators': [10, 20, 30, 40, 50]})

In [33]:
print(reg.score(X_test, y_test))
print(reg.best_params_)

-0.014560758504354565
{'max_features': 0.8, 'n_estimators': 40}


### Стэкинг

**Стэкинг**: на основе обученных базовых алгоритмов строится новый *метаалгоритм*, для вывода предсказаний, основанных на множественных предсказаниях, возвращаемых этими базовыми алгоритмами

In [42]:
estimators = [
    ('dtr_1', DecisionTreeRegressor(random_state=0)),
    ('dtr_2', DecisionTreeRegressor(random_state=0))
]

In [43]:
stacking_reg = StackingRegressor(estimators, final_estimator=RandomForestRegressor(random_state=0))

stacking_reg.fit(X_train, y_train)

StackingRegressor(estimators=[('dtr_1', DecisionTreeRegressor(random_state=0)),
                              ('dtr_2', DecisionTreeRegressor(random_state=0))],
                  final_estimator=RandomForestRegressor(random_state=0))

In [44]:
print(stacking_reg.score(X_test, y_test))

-0.18009016329532268


In [46]:
estimators = [
    ('dtr', DecisionTreeRegressor(random_state=0)),
    ('rf', RandomForestRegressor(random_state=0))
]

In [47]:
stacking_reg = StackingRegressor(estimators, final_estimator=RandomForestRegressor(random_state=0))

stacking_reg.fit(X_train, y_train)

StackingRegressor(estimators=[('dtr_1', DecisionTreeRegressor(random_state=0)),
                              ('dtr_2', RandomForestRegressor(random_state=0))],
                  final_estimator=RandomForestRegressor(random_state=0))

In [48]:
print(stacking_reg.score(X_test, y_test))

-0.09203335588390416


### Градиентный бустинг

**Градиентный бустинг**: градиентный бустинг сводит задачу к градиентному спуску: на каждой итерации мы подгоняем слабого ученика к антиградиенту текущей ошибки подбора по отношению к текущей модели ансамбля

In [52]:
grad_reg = GradientBoostingRegressor()

In [54]:
params = {
    'loss': ['ls', 'lad', 'huber'],
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200, 300]
}

In [57]:
%%time
reg = GridSearchCV(grad_reg, params)

reg.fit(X_train, y_train)

Wall time: 26min 20s


GridSearchCV(estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.01, 0.1],
                         'loss': ['ls', 'lad', 'huber'],
                         'n_estimators': [100, 200, 300]})

In [58]:
print(reg.score(X_test, y_test))
print(reg.best_params_)

0.0893249989424405
{'learning_rate': 0.1, 'loss': 'ls', 'n_estimators': 300}


## Вывод

Наилучшим образом себя показал градиентный бустинг, однако, он обучался в разы дольше других композиций

## CatBoost

Используем для решения задачи CatBoostRegressor

In [9]:
from catboost import CatBoostRegressor

In [26]:
cat_model = CatBoostRegressor(iterations=1500,
                              task_type="GPU",
                              devices='0:1')

In [27]:
%%time
cat_model.fit(X_train, y_train,
             verbose=False)

Wall time: 26.3 s


In [28]:
cat_rmse = cat_model.score(X_test, y_test)
cat_rmse

0.06532193838194789

CatBoost прекрасен! Время обучения намного меньеш, а RMSE выше. 